In [1]:
import sys, os
sys.path.append(os.pardir)
sys.path.append(os.curdir)
import axon
import axon.functions as F
from axon import DataLoader
from axon.models import MLP

max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = axon.datasets.MNIST(train=True)
test_set = axon.datasets.MNIST(train=False)
train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, hidden_size, 10), activation=F.relu)
optimizer = axon.optimizers.Adam().setup(model)
optimizer.add_hook(axon.optimizers.WeightDecay(1e-4))  # Weight decay

print('GPU: {}'.format(axon.core.gpu_enable))

for epoch in range(max_epoch):
    sum_loss, sum_acc = 0, 0

    # 训练
    for x, t in train_loader:
        y = model(x)
        loss = F.softmax_cross_entropy(y, t) # 交叉熵误差
        acc = F.accuracy(y, t) # 计算精度
        model.cleargrads()
        loss.backward() # 反向传播
        optimizer.update() # 更新参数

        sum_loss += float(loss.data) * len(t) # 累计损失
        sum_acc += float(acc.data) * len(t) # 累计精度

    print('epoch: {}'.format(epoch+1))
    print('train loss: {}, accuracy: {}'.format(
        sum_loss / len(train_set), sum_acc / len(train_set)))

    # 测试
    sum_loss, sum_acc = 0, 0
    with axon.no_grad():
        for x, t in test_loader:
            y = model(x)
            loss = F.softmax_cross_entropy(y, t)
            acc = F.accuracy(y, t)
            sum_loss += float(loss.data) * len(t)
            sum_acc += float(acc.data) * len(t)

    print('test loss: {}, accuracy: {}'.format(
        sum_loss / len(test_set), sum_acc / len(test_set)))

GPU: True
epoch: 1
train loss: 0.1917829109635204, accuracy: 0.9419166666666666
test loss: 0.1047103403962683, accuracy: 0.9664
epoch: 2
train loss: 0.08020634535700083, accuracy: 0.9749166666666667
test loss: 0.07306548073829618, accuracy: 0.9784
epoch: 3
train loss: 0.05478894843099018, accuracy: 0.9822833333333333
test loss: 0.0754341715888586, accuracy: 0.9757
epoch: 4
train loss: 0.046337428902237056, accuracy: 0.9849666666666667
test loss: 0.0667539263695653, accuracy: 0.9773
epoch: 5
train loss: 0.03971771081715512, accuracy: 0.9869666666666667
test loss: 0.07553065540909301, accuracy: 0.9764
